In [8]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras import models, layers
import tensorflow as tf

import tensorflow.experimental.numpy as tnp

tnp.experimental_enable_numpy_behavior()

from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

print(tf.config.list_physical_devices())
GPU = '/GPU:0'

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
CSV_LINK = 'https://raw.githubusercontent.com/Genia163/Datasets-for-lab/main/9_table_NN/data_banknote_authentication.csv'
MIN_ACCURACY = 0.737
MAX_ACCURACY = 0.851
opt = tf.optimizers.Adam(1e-5)

In [10]:
csv_set = pd.read_csv(CSV_LINK,
                      names=[0, 1, 2, 3, 4],
                      engine="python",
                      skiprows=1)

header = csv_set.take([0])
csv_set = csv_set.drop(0)

X = csv_set.drop(4, axis=1).to_numpy()
y = csv_set.take([4], axis=1)

y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=40)

In [18]:
def make_embedding():
    model = models.Sequential(name='banknotes')

    model.add(Input(shape=(4,), name='data_input'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='sigmoid'))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [19]:
model = make_embedding()
model.summary()

Model: "banknotes"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                320       
                                                                 
 dense_9 (Dense)             (None, 1024)              66560     
                                                                 
 dense_10 (Dense)            (None, 512)               524800    
                                                                 
 dense_11 (Dense)            (None, 64)                32832     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 2)                 130       
                                                                 
Total params: 624,642
Trainable params: 624,642
Non-train

In [20]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, model=model)

In [21]:
EPOCHES = 0
with tf.device(GPU):
    while True:
        EPOCHES += 1
        model.fit(X_train, y_train, epochs=EPOCHES, batch_size=1)
        precision = model.evaluate(X_train, y_train, verbose=0)[1]
        if (precision > MIN_ACCURACY) & (precision < MAX_ACCURACY):
            break
print(f'Accuracy: {precision}')

822/822 [==============================] - 3s 3ms/step - loss: 0.5212 - accuracy: 0.7591
Epoch 1/2
822/822 [==============================] - 2s 2ms/step - loss: 0.3445 - accuracy: 0.8893
Epoch 2/2
822/822 [==============================] - 2s 2ms/step - loss: 0.2625 - accuracy: 0.9453
Epoch 1/3
822/822 [==============================] - 2s 2ms/step - loss: 0.2060 - accuracy: 0.9623
Epoch 2/3
822/822 [==============================] - 2s 2ms/step - loss: 0.1604 - accuracy: 0.9672
Epoch 3/3
822/822 [==============================] - 2s 2ms/step - loss: 0.1480 - accuracy: 0.9562
Epoch 1/4
822/822 [==============================] - 2s 2ms/step - loss: 0.1150 - accuracy: 0.9732
Epoch 2/4
822/822 [==============================] - 2s 3ms/step - loss: 0.0936 - accuracy: 0.9757
Epoch 3/4
730/822 [=========================>....] - ETA: 0s - loss: 0.0882 - accuracy: 0.9808

KeyboardInterrupt: 

In [15]:
test_labels = model.predict(X_test, verbose=0)
t_l, t_a = model.evaluate(X_test, test_labels, verbose=0)
print(f'Loss: {t_l:.2f} \nAccuracy: {t_a:.2f}')

Loss: 0.67 
Accuracy: 1.00


In [16]:
def get_classified(data):
    classes = np.empty(len(data), dtype='int')
    for i, bit in enumerate(data):
        if bit[0] > bit[1]:
            classes[i] = 0
        else:
            classes[i] = 1
    return classes

In [17]:
final_table = pd.DataFrame(test_labels)
classifier = get_classified(final_table.to_numpy())

y_test_uncategorical = get_classified(y_test)
final_table['y_test'] = y_test_uncategorical
final_table['predicted'] = classifier

final_table

,0,1,y_test,predicted
0,0.485579,0.490660,1,1
1,0.529878,0.431741,0,0
2,0.503127,0.508260,1,1
3,0.591900,0.327629,0,0
4,0.553460,0.417363,0,0
...,...,...,...,...
544,0.505447,0.513131,1,1
545,0.626722,0.317436,0,0
546,0.544170,0.420203,0,0
547,0.529018,0.396701,0,0
